In [14]:
import torch
from detectron2._C import box_iou_rotated
from detectron2._C import nms_rotated

In [17]:
boxes1, boxes2 = torch.rand(100, 5), torch.rand(20000, 5)
iou = box_iou_rotated(boxes1, boxes2)
assert iou.shape == (100, 20000)

In [18]:
scores = torch.rand(100) 
iou_threshold = 0.5
output = nms_rotated(boxes1, scores, iou_threshold) 
output.shape

torch.Size([71])